# Data Reshaping + Tidy Data

- [Data Reshaping](#Data-Reshaping)
    - [Melt Example](#Melt-Example)
    - [Pivot Table Example](#Pivot-Table-Example)
- [Tidy Data](#Tidy-Data)
    - [One Column with Multiple Variables](#One-Column-with-Multiple-Variables)
    - [One Variable in Multiple Columns](#One-Variable-in-Multiple-Columns)
    - [Multiple vars in 2 columns](#Multiple-vars-in-2-columns)
    - [Another gnarly example](#Another-gnarly-example)
    - [A More Complex Example](#A-More-Complex-Example)

In [2]:
import pandas as pd
import numpy as np

## Data Reshaping

Tidy != Clean

- **long** data has many rows and few columns
- **wide** data has many columns
- a **melt** takes the data from wide to long
- a **spread**, or **pivot** takes the data from long to wide
- a **transpose** rotates the dataframe 90 degrees

### Melt Example

In [10]:
np.random.seed(123)

# simple data for demonstration
df = pd.DataFrame({
    'a': np.random.randint(1, 11, 3),
    'b': np.random.randint(1, 11, 3),
    'c': np.random.randint(1, 11, 3),
    'x': np.random.randint(1, 11, 3),
    'y': np.random.randint(1, 11, 3),
    'z': np.random.randint(1, 11, 3),    
})
df

,a,b,c,x,y,z
0,3,2,7,2,1,5
1,3,4,2,10,10,1
2,7,10,1,1,4,1


Different ways of using `.melt`:

In [15]:
# id_vars: keep these columns as is
# value_vars: combine these columns (combine *all* columns if omitted)
df.melt(id_vars=['a', 'b'], value_vars=['x', 'y'], var_name='original_column_name', value_name='original_value')

,a,b,original_column_name,original_value
0,3,2,x,2
1,3,4,x,10
2,7,10,x,1
3,3,2,y,1
4,3,4,y,10
5,7,10,y,4


In [ ]:
# df.melt()
# df.melt(id_vars='a')
# df.melt(id_vars='x')
# df.melt(id_vars=['a', 'b'])
# df.melt(value_vars=['x', 'y', 'z'])
# df.melt(id_vars=['a', 'b'], value_vars=['x', 'y'], var_name='foo', value_name='bar')

### Pivot Table Example

Turns the unique values in a single column into columns themselves.

In [17]:
np.random.seed(123)
df = pd.DataFrame({
    'group': np.random.choice(['A', 'B', 'C'], 20),
    'subgroup': np.random.choice(['one', 'two'], 20),
    'x': np.random.randn(20),
})
df.head()

,group,subgroup,x
0,C,two,0.737369
1,B,one,1.490732
2,C,two,-0.935834
3,C,one,1.175829
4,A,one,-1.253881


In [22]:
# values, rows, columns
df.pivot_table('x', 'subgroup', 'group')

group,A,B,C
subgroup,,,
one,-0.710190,-0.669245,0.423405
two,-1.771533,-0.545111,0.087422


## Tidy Data

Tidy Data Characteristics:

- data is tabular, i.e. made up of rows and columns
- there is one value per cell
- each variable is a column
- each observation is a row

General Ideas

- If the units are the same, maybe they should be in the same column
- If one column has measurements of different units, it should be spread out
- Should you be able to groupby some of the columns? combine them
- Can I pass this data to seaborn?
- Can we ask interesting questions and answer them with a group by? I.e. generally we **don't** want to be taking row or column averages.

For the rest of this lesson, we'll look at data that is **not** tidy.

### One Column with Multiple Variables

In [24]:
df = pd.DataFrame({
    'name': ['Sally', 'Jane', 'Billy', 'Suzy'],
    'pet': ['dog: max', 'dog: buddy', 'cat: grizabella', 'hamster: fred']
})
df

,name,pet
0,Sally,dog: max
1,Jane,dog: buddy
2,Billy,cat: grizabella
3,Suzy,hamster: fred


In [38]:
pet_name_and_species = df.pet.str.split(': ', expand=True)
pet_name_and_species.columns = ['pet_species', 'pet_name']
tidy_df = pd.concat([df, pet_name_and_species], axis=1).drop(columns='pet')
tidy_df

,name,pet_species,pet_name
0,Sally,dog,max
1,Jane,dog,buddy
2,Billy,cat,grizabella
3,Suzy,hamster,fred


### One Variable in Multiple Columns

In [41]:
np.random.seed(123)

df = pd.DataFrame(
    np.random.uniform(60, 100, (4, 4)),
    columns=['Sally', 'Jane', 'Billy', 'Suzy'],
    index = pd.Index(['spelling', 'math', 'reading', 'nuclear physics'], name='subject')
).round(1).reset_index()
df

,subject,Sally,Jane,Billy,Suzy
0,spelling,87.9,71.4,69.1,82.1
1,math,88.8,76.9,99.2,87.4
2,reading,79.2,75.7,73.7,89.2
3,nuclear physics,77.5,62.4,75.9,89.5


In [46]:
df = df.set_index('subject').transpose().reset_index()

In [51]:
df.columns.name = ''
df = df.rename(columns={'index': 'student'})

| student | subject | grade |
| ------- | ------- | ---- |
| Sally | spelling | 87.9 |
| Sally | math | 88.8 |

In [56]:
df = df.melt(id_vars=['student'], var_name='subject', value_name='grade')
df

,student,subject,grade
0,Sally,spelling,87.9
1,Jane,spelling,71.4
2,Billy,spelling,69.1
3,Suzy,spelling,82.1
4,Sally,math,88.8
5,Jane,math,76.9
6,Billy,math,99.2
7,Suzy,math,87.4
8,Sally,reading,79.2
9,Jane,reading,75.7


- what is the average spelling grade?
- What is Jane's average grade?

Sometimes it is desirable to "untidy" the data for quick analysis / visualization. E.g. spread subject out to columns, students as rows.

In [58]:
df.groupby('student').mean()

,grade
student,
Billy,79.475
Jane,71.600
Sally,83.350
Suzy,87.050


### Multiple vars in 2 columns

- "incorrect melt"

In [68]:
df = pd.read_csv('./untidy-data/gapminder1.csv')
# rows -- what defines one observation?
df = df.pivot_table('measurement', ['year', 'country'], 'measure')
df = df.reset_index()
df.columns.name = ''
df.sort_values(['country', 'year'])

,year,country,fertility,life_expect,pop
0,1955,Afghanistan,7.7000,30.332,8891209.0
63,1960,Afghanistan,7.7000,31.997,9829450.0
126,1965,Afghanistan,7.7000,34.020,10997885.0
189,1970,Afghanistan,7.7000,36.088,12430623.0
252,1975,Afghanistan,7.7000,38.438,14132019.0
...,...,...,...,...,...
440,1985,Venezuela,3.6485,70.190,16997509.0
503,1990,Venezuela,3.2500,71.150,19325222.0
566,1995,Venezuela,2.9415,72.146,21555902.0
629,2000,Venezuela,2.7230,72.766,23542649.0


### Another gnarly example

In [100]:
df = pd.read_csv('untidy-data/gapminder2.csv')
df = df.melt(id_vars='country')
df['year'] = df.variable.str[-4:]
df['measure'] = df.variable.str[:-5]
df = df.drop(columns='variable')
df = df.pivot_table('value', ['country', 'year'], 'measure')
df = df.reset_index()
df.columns.name = ''
df

,country,year,life_expect,pop
0,Afghanistan,1955,30.332,8891209.0
1,Afghanistan,1960,31.997,9829450.0
2,Afghanistan,1965,34.020,10997885.0
3,Afghanistan,1970,36.088,12430623.0
4,Afghanistan,1975,38.438,14132019.0
...,...,...,...,...
688,Venezuela,1985,70.190,16997509.0
689,Venezuela,1990,71.150,19325222.0
690,Venezuela,1995,72.146,21555902.0
691,Venezuela,2000,72.766,23542649.0


### A More Complex Example

In [116]:
sales = pd.read_csv('./untidy-data/sales.csv')
sales = sales.melt(id_vars='Product')

year_and_measure_df = sales.variable.str.split(' ', expand=True)
year_and_measure_df.columns = ['year', 'measure']

df = pd.concat([sales, year_and_measure_df], axis=1)
df = df.drop(columns='variable')

df = df.pivot_table('value', ['Product', 'year'], 'measure')

df = df.reset_index()
df.columns.name = ''
df

,Product,year,PPU,Sales
0,A,2016,5,673
1,A,2017,7,231
2,A,2018,9,173
3,B,2016,3,259
4,B,2017,5,748
5,B,2018,8,186
6,C,2016,3,644
7,C,2017,5,863
8,C,2018,5,632
9,D,2016,9,508


1. Download the file from google classroom / slack

    google classroom instructions

    1. Click on the three dots in the upper right corner
    1. Choose "open in new window"
    1. Now there should be a download button in the upper right of the web page

1. Extract the archive and move it into the right place

    ```
    cd ~/Downloads
    tar xzvf untidy-data.tgz
    mv untidy-data ~/codeup-data-science/classification-exercises
    ```
    
1. Add csv files to your gitignore

    Make sure you have this line in your `.gitignore`
    
    ```
    *.csv
    ```